In [1]:
import cv2
import numpy as np
import os
from tqdm import tqdm
from IPython.display import clear_output
import random

In [2]:
label__dict = {
    0:"otomobil",
    1:"motosiklet",
    2:"otobüs",
    3:"kamyon",
    4:"diger",
    5:"lokomotif",
    6:"vagon",
    7:"tramvay",
    8:"monoray",
    9:"feniküler",
    10:"deniz tasiti",
    11:"insan",
    12:"uap",
    13:"uai",
}

In [3]:
import tkinter as tk
root = tk.Tk()
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()
frame_display_size = (int(screen_width*0.8),int(screen_height*0.8))

color1 = (100,100,255)
color2 = (0,0,0)

font = cv2.FONT_HERSHEY_DUPLEX


In [4]:
def path_sequence(root,*directories):
    seq = root
    for directory in directories:
        seq += f"/{directory}"
    return seq


def format_line(line):
    words = line.split(" ")
    return int(words[0]),float(words[1]),float(words[2]),float(words[3]),float(words[4])

def format_problem_line(line):
    words = line.split(" ")
    return words[1],words[3],words[5][:-1]

def remove_file(path):
    if os.path.exists(path):
        os.remove(path)
    else:
        print("belirtilen yolda dosya bulunamadı")
        

In [5]:
frame_display_size = (int(screen_width*0.8),int(screen_height*0.8))

color1 = (100,100,255)
color2 = (0,0,0)

font = cv2.FONT_HERSHEY_DUPLEX

In [6]:
remove_all = True

In [7]:
## problemleri okuyalım
p = open("problems.txt")
problems = p.readlines()
p.close()

cleaning_indexes = []
print(len(problems))



##  problemleri dolaşalım

root = "./datasets/visdrone/detect" ## doldurulacak
images_path = path_sequence(root,"images")
labels_path = path_sequence(root,"labels")

shown_counter = 0

for problem_idx, problem in tqdm(enumerate(problems)):
    cat,im_name,txt_name =format_problem_line(problem)
    
    im_path = path_sequence(images_path,cat,im_name)
    lbl_path = path_sequence(labels_path,cat,txt_name)
    
    print(problem_idx,cat,im_path,lbl_path)
    
    if remove_all != True:
        # frame
        original_image = cv2.imread(im_path)
        image = original_image.copy()

        # etiketler
        f = open(lbl_path,"r")
        lines = f.readlines()
        f.close()
        for line in lines:
            label, norm_center_x, norm_center_y, norm_width, norm_height = format_line(line)

            width = image.shape[1] * norm_width
            height = image.shape[0] * norm_height
            x1 = int(image.shape[1]*norm_center_x) - int(width/2)
            x2 = int(image.shape[1]*norm_center_x) + int(width/2)
            y1 = int(image.shape[0]*norm_center_y) - int(height/2)
            y2 = int(image.shape[0]*norm_center_y) + int(height/2)

            # etiketleri çizdirelim
            cv2.putText(image,f"{label__dict[int(label)]}",(x1+5,y1+15),font,0.55,color1)
            cv2.rectangle(image,(x1,y1),(x2,y2),color1,2)
            print(f"{label}:{label__dict[int(label)]}",norm_center_x,norm_center_y,norm_height,norm_height)


         ## ekranda göster 
        image_ = cv2.resize(image,frame_display_size)

        ## bilgi yazıları
        cv2.putText(image_,str(shown_counter),(10,40),font,1.2,color2)
        for i,text in enumerate(["space:atla","p:kaldir","q:bitir"]):
            cv2.putText(image_,text,(10,image_.shape[0]-35*i-25),font,0.8,color2)

        window_name = f"{cat}  -  {im_name}  -  {txt_name}"

        cv2.namedWindow(window_name)
        cv2.moveWindow(window_name,20,20)
        cv2.imshow(window_name,image_)

        key = cv2.waitKey(0)
        if key == ord("q"):
            cv2.destroyAllWindows()
            clear_output()
            break
        elif key == ord("p"):
            cleaning_indexes.append(problem_idx)
            clear_output()
            cv2.destroyAllWindows()

            remove_file(im_path)
            remove_file(lbl_path)
            continue
        elif key == ord(" "):
            cv2.destroyAllWindows()
            clear_output()
            shown_counter+=1 
            continue
    else:
        remove_file(im_path)
        remove_file(lbl_path)
        
        
## tamamlanan problemleri kaldıralım ve yazalım ve loglayalım
logs = []
for c_index in cleaning_indexes[::-1]:
    log = problems.pop(c_index)
    logs.append(log)
    
p = open("problems.txt","w")
p.writelines(problems)
p.close()

p = open("handled_problems.txt","a")
p.writelines(logs)
p.close()

15


15it [00:00, 15004.66it/s]

0 train ./datasets/visdrone/detect/images/train/9999955_00000_d_0000284.jpg ./datasets/visdrone/detect/labels/train/9999955_00000_d_0000284.txt
belirtilen yolda dosya bulunamadı
belirtilen yolda dosya bulunamadı
1 train ./datasets/visdrone/detect/images/train/9999987_00000_d_0000006.jpg ./datasets/visdrone/detect/labels/train/9999987_00000_d_0000006.txt
belirtilen yolda dosya bulunamadı
belirtilen yolda dosya bulunamadı
2 train ./datasets/visdrone/detect/images/train/9999955_00000_d_0000121.jpg ./datasets/visdrone/detect/labels/train/9999955_00000_d_0000121.txt
belirtilen yolda dosya bulunamadı
belirtilen yolda dosya bulunamadı
3 train ./datasets/visdrone/detect/images/train/9999999_00271_d_0000118.jpg ./datasets/visdrone/detect/labels/train/9999999_00271_d_0000118.txt
belirtilen yolda dosya bulunamadı
belirtilen yolda dosya bulunamadı
4 train ./datasets/visdrone/detect/images/train/9999964_00000_d_0000074.jpg ./datasets/visdrone/detect/labels/train/9999964_00000_d_0000074.txt
belirtil

In [37]:
print(len(problems))
print(problems)
print()
print(logs)

4
['aerialMaritime train image: DJI_0260_JPG.rf.8a65e206c6c29ab774f8240f83387594.jpg txt: DJI_0260_JPG.rf.8a65e206c6c29ab774f8240f83387594.txt\n', 'aerialMaritime train image: DJI_0260_JPG.rf.d96e4fa73edbce6fcfae2efa29bc0a30.jpg txt: DJI_0260_JPG.rf.d96e4fa73edbce6fcfae2efa29bc0a30.txt\n', 'aerialMaritime train image: DJI_0261_JPG.rf.4a4237e833afa8352951c7cfb0d1c2f4.jpg txt: DJI_0261_JPG.rf.4a4237e833afa8352951c7cfb0d1c2f4.txt\n', 'aerialMaritime train image: DJI_0261_JPG.rf.59b70123c75e6578da7f9af8e540f953.jpg txt: DJI_0261_JPG.rf.59b70123c75e6578da7f9af8e540f953.txt\n']

[]


In [27]:
lbl_path

'./dataset Aerial Maritime/detect/labels/train/DJI_0258_JPG.rf.c9d34732d58ef73927f5d98d4045d420.tx'